In [21]:
import pickle

class Animal :
    def __init__(self, number_of_paws, color) :
        self.number_of_paws = number_of_paws
        self.color = color
        
    
class Sheep(Animal) :
    def __init__(self, color) :
        Animal.__init__(self, 4,color)
        

lary = Sheep("White")
lary.color

# di pickle 
mypickle_lary = pickle.dumps(lary)
print('\nWould u like to pickle ? he she is')
print(mypickle_lary)

#unpickle larry
dolly = pickle.loads(mypickle_lary)

#dolly and lary are different object
dolly.color = "Black"

print("\nLary is {}".format(lary.color))
print('\nDolly is {}'.format(dolly.color))


Would u like to pickle ? he she is
b'\x80\x03c__main__\nSheep\nq\x00)\x81q\x01}q\x02(X\x0e\x00\x00\x00number_of_pawsq\x03K\x04X\x05\x00\x00\x00colorq\x04X\x05\x00\x00\x00Whiteq\x05ub.'

Lary is White

Dolly is Black


In [4]:
import pickle

class my_zen_class :
    number_of_meditations = 0
    
    def __init__(self, name) :
        self.number_of_mediations = 0
        self.name = name
        
    def meditate(self) :
        self.number_of_meditations += 1
        
    def __getstate__(self) :
        # You will never get the Buddha state if you count 
       # meditations, so 
       # don't pickle this counter, the next time you will just 
       # start meditating from scratch :)
        state = self.__dict__.copy()
        
        del state['number_of_meditations']
    
        return state
    
    def __setstate__(self, state) :
        # this method is called when you are going to 
       # unpickle the class,
       # if you need some initialization after the 
       # unpickling you can do it here.
        self.__dict__.update(state)
        
    
# i start meditating
my_zen_object = my_zen_class("Dave")
for i in range (100) :
    my_zen_object.meditate()
    
    
# now i pickel my meditation experience
print("i am {} and i have meditated {} times". format(my_zen_object.name, my_zen_object.number_of_meditations))

mypickled_zen_object = pickle.dumps(my_zen_object)
my_zen_object = None

# now i get my experience back
mynew_zen_object = pickle.loads(mypickled_zen_object)

# As expected, the number_of_meditations property 
# has not been restored because it hasn't been pickled
print('i am {}, and i dont have a beginner mind because i have meditated {} times'.format(mynew_zen_object.name, mynew_zen_object.number_of_meditations))

i am Dave and i have meditated 100 times
i am Dave, and i dont have a beginner mind because i have meditated 0 times


In [15]:
import pickle
import sqlite3
from collections import namedtuple

MemoRecord = namedtuple("MemoRecord", "key, task")

class DBPickler(pickle.Pickler) :
    def persistent_id(self, obj) :
        if isinstance(obj, MemoRecord) :
            return ("MemoRecord", obj.key)
        else :
            return None
        

class DBUnpickler(pickle.Unpickler) :
    def __init__(self, file, connection) :
        super().__init__(file) 
        self.connection = connection
        
    def persistent_load(self, pid) :
        cursor = self.connection.cursor()
        type_tag, key_id = pid
        
        if type_tag == "MemoRecord" :
            cursor.execute("SELECT * from memos WHERE key=?", (str(key_id),))
            key, task = cursor.fetchone()
            return MemoRecord(key, task)
        else :
            raise pickle.UnpicklingError("unsupported persistent object")
            

def main() :
    import io
    import pprint
    
    # initialized and populate database
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE memos(key INTEGER PRIMARY KEY, task TEXT)")
    tasks = (
        'give food to fish',
        'prepare group meeting',
        'fight with a zebra',
    )
    
    for task in tasks :
        cursor.execute("INSERT into memos VALUES(NULL, ?)", (task,))
        
    #fetch the cuursor to be picked
    cursor.execute("SELECT * from memos")
    memos =[MemoRecord(key, task) for key, task in cursor]
    
    #save the record
    file = io.BytesIO()
    DBPickler(file).dump(memos)
    
    print("Picked records :")
    pprint.pprint(memos)
    
    #update record
    cursor.execute("UPDATE memos SET task='learn italian' WHERE key=1")
    
    #load the record from the pickle data stream
    file.seek(0)
    memos = DBUnpickler(file, conn).load()
    
    print("unpickle record")
    pprint.pprint(memos)
    
if __name__ == '__main__' :
    main()

Picked records :
[MemoRecord(key=1, task='give food to fish'),
 MemoRecord(key=2, task='prepare group meeting'),
 MemoRecord(key=3, task='fight with a zebra')]
unpickle record
[MemoRecord(key=1, task='learn italian'),
 MemoRecord(key=2, task='prepare group meeting'),
 MemoRecord(key=3, task='fight with a zebra')]


In [27]:
import pickle
class TextReader :
    def __init__(self, filename) :
        self.filename = filename
        self.file = open(filename)
        self.lineno = 0
        
    def readline(self) :
        self.lineno += 1
        line = self.file.readline()
        if not line :
            return None
        if line.endswith('\n') :
            line = line[:-1]
        return("{} {}".format(self.lineno, line))
    
    def __getstate__(self) :
        # Copy the object's state from self.__dict__ which contains
        # all our instance attributes. Always use the dict.copy()
        # method to avoid modifying the original state.
        state = self.__dict__.copy()
        #remove the unpickle entries
        del state['file']
        return state
    
    def __setstate__(self, state) :
        # restore instance attribute 
        self.__dict__.update(state)
        # Restore the previously opened file's state. To do so, we need to
        # reopen it and read from it until the line count is restored.
        file = open(self.filename)
        for _ in range(100) :
            file.readline()
        #finally save the file
        self.file = file
        
reader=TextReader('hello.txt')
reader.readline()
print('\n')
reader.readline()
print('\n')
reader.readline()
new_reader = pickle.loads(pickle.dumps(reader))
new_reader.readline()
        

In [26]:
import pickle
class TextReader :
    def __init__(self, filename) :
        self.filename = filename
        self.file = open(filename)
        self.lineno = 0
        
    def readline(self) :
        self.lineno += 1
        line = self.file.readline()
        if not line :
            return None
        if line.endswith('\n') :
            line = line[:-1]
        return("{} {}".format(self.lineno, line))
    
    def __getstate__(self) :
        # Copy the object's state from self.__dict__ which contains
        # all our instance attributes. Always use the dict.copy()
        # method to avoid modifying the original state.
        state = self.__dict__.copy()
        #remove the unpickle entries
        del state['file']
        return state
    
    def __setstate__(self, state) :
        # restore instance attribute 
        self.__dict__.update(state)
        # Restore the previously opened file's state. To do so, we need to
        # reopen it and read from it until the line count is restored.
        file = open(self.filename)
        for _ in range(100) :
            file.readline()
        #finally save the file
        self.file = file
        
reader=TextReader('hello.txt')
reader.readline()
print('\n')
reader.readline()
print('\n')
reader.readline()
new_reader = pickle.loads(pickle.dumps(reader))
new_reader.readline()

In [7]:
import builtins
import io
import pickle

safe_builtins = {
    'range',
    'complex',
    'set',
    'frozenset',
    'slice',
}

class RestrictedUnpickled(pickle.Unpickler) :
    def find_class(self, module, name) :
        if(module == builtins and name in safe_builtins) :
            return getattr(builtins, name)
        raise pickle.UnpicklingError("global {} {} is forbidden".format(module,name))
        
def restricted_load(s) :
    """Helper function analogous to pickle.loads()."""
    return RestrictedUnpickled(io.BytesIO(s)).load()

restricted_load(pickle.dumps([1,2, range(15)]))

UnpicklingError: global builtins range is forbidden

In [15]:
import pickle

data = {
    'a' : [1,2.0,3,4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

with open('data.pickle', 'wb') as f :
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    
with open('data.pickle', 'rb') as f :
    data = pickle.load(f)